Author: Kyle Herbruger
Date: 10/17/2023

Program loads in two .csv files, with the xy information from the OpenCV hand tracker. Then plots them, and converts to a gif.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import time

from scipy.io.wavfile import read

In [2]:
class plotGenerator():
    def __init__(self, dataX, dataY, dataZ, song = '', side = '', leng=0):
        self.dataX = dataX
        self.dataY = dataY
        self.dataZ = dataZ
        self.song = song
        self.side = side
                
        #dataY = dataY - dataY.max()
        dataZ = dataZ * -1
    
        badFrameCount = 0
        # bounds for Z axis
        zMax = max(np.max(dataZ), abs(np.min(dataZ)))
        
        # Clock to show user on time progress/left/taken
        clock_start = time.time()
        
        # Generating plots
        for ia in range(leng):
            # clrZ no used, but used to make plot points size/color vary with z-axis
            clrZ = np.array(dataZ[ia])/zMax
            ptClr = 'ro-'
            # Checks if data is from frame where hand was not detected (all zero)
            if np.sum(dataX[ia,0:21]) == 0:
                # Adjusts plot to use previous frame data/change hand color
                badFrameCount += 1
                ptClr = 'mo--'
                dataX[ia, 0:21] = dataX[ia - 1, 0:21]
                dataY[ia, 0:21] = dataY[ia - 1, 0:21]
            # Plots
            plt.plot(dataX[ia,0:5],dataY[ia,0:5], ptClr)
            plt.plot(dataX[ia,5:9],dataY[ia,5:9], ptClr)
            plt.plot(dataX[ia,9:13],dataY[ia,9:13], ptClr)
            plt.plot(dataX[ia,13:17],dataY[ia,13:17], ptClr)
            plt.plot(dataX[ia,17:21],dataY[ia,17:21], ptClr)
            plt.plot([dataX[ia,0],dataX[ia,5],dataX[ia,9],dataX[ia,13],dataX[ia,17],dataX[ia,0]],
                    [dataY[ia,0],dataY[ia,5],dataY[ia,9],dataY[ia,13],dataY[ia,17],dataY[ia,0]], ptClr)
            #plt.scatter(dataX[ia,0:21],dataY[ia,0:21], s=clrZ[0:21]*30+25, c=clrZ[0:21],cmap='Oranges')
            
            
            ptClr = 'bo-'
            if np.sum(dataX[ia,22:42]) == 0:
                # Adjusts plot to use previous frame data/change hand color
                badFrameCount += 1
                ptClr = 'co--'
                dataX[ia, 21:42] = dataX[ia - 1, 21:42]
                dataY[ia, 21:42] = dataY[ia - 1, 21:42]
            plt.plot(dataX[ia,21:26],dataY[ia,21:26], ptClr)
            plt.plot(dataX[ia,26:30],dataY[ia,26:30], ptClr)
            plt.plot(dataX[ia,30:34],dataY[ia,30:34], ptClr)
            plt.plot(dataX[ia,34:38],dataY[ia,34:38], ptClr)
            plt.plot(dataX[ia,38:42],dataY[ia,38:42], ptClr)
            plt.plot([dataX[ia,21],dataX[ia,26],dataX[ia,30],dataX[ia,34],dataX[ia,38],dataX[ia,21]],
                    [dataY[ia,21],dataY[ia,26],dataY[ia,30],dataY[ia,34],dataY[ia,38],dataY[ia,21]], ptClr)
            #plt.scatter(dataX[ia,22:42],dataY[ia,22:42], s=clrZ[22:42]*30+25, c=clrZ[22:42],cmap='Purples')
            
            # Inverting Y-axis and removing plot markers, and setting constant xlim/ylim
            plt.ylim(dataY[ia].max() + 100, dataY[ia].min() - 100)
            plt.xlim(0, dataX[ia].max() + 100)
            plt.xticks([])
            plt.yticks([])
            # saving
            plt.savefig(f'./TriCam/{song.upper()}/img/img_{song}{ia}_{side}.png',transparent=False,facecolor='white')
            plt.close()
           
            # Updating on time taken
            if (ia % 500) == 0:
                # Updates user on time taken, progress, and eta for completion.
                time_ran = time.time() - clock_start
                perc_done = ((ia + 1)/leng)*100
                time_left_guess = time_ran/perc_done * (100 - perc_done)
                print(f'Time: {int(time_ran/60)}:{int(time_ran%60):2} ({perc_done:.2f}%)')
                print(f'ETA: {int(time_left_guess/60)}:{int(time_left_guess%60)} [{ia}/{leng}]')
        
        # making .gifs
        print('Schemeing...')
        print(f'{badFrameCount} bad frames ({(badFrameCount/leng)*100:.2f})')
        time_ran = time.time() - clock_start
        print(f'Time: {int(time_ran/60)}:{int(time_ran%60):2}')

In [3]:
def gifMaker(song, side, leng):
    song = song
    side = side
    frames = []
    for ia in range(leng):
         # adding to frame stack for .gif making
        imageb = imageio.v2.imread(f'./TriCam/{song.upper()}/img/img_{song}{ia}_{side}.png')
        frames.append(imageb)
    imageio.mimsave(f'./TriCam/gifs/{song}_{side}_30.gif',
                   frames,
                   fps=30)
    print('Scheme complete >:)')

        

In [4]:
def main(song, side):
    fileLoc = f'./TriCam/{song.upper()}/Cleaned/{song}_cut_{side}_combo'

    dataX = np.loadtxt(f'{fileLoc}_X.csv', delimiter=",", dtype=int)
    dataY = np.loadtxt(f'{fileLoc}_Y.csv', delimiter=",", dtype=int)
    dataZ = np.loadtxt(f'{fileLoc}_Z.csv', delimiter=",", dtype=int)

    # Processors
    leng = len(dataX) - 10
    plotGenerator(dataX, dataY, dataZ, song, side, leng)
    gifMaker(song, side, leng)
    #diffProcessor(dataX, dataY, dataZ, song)

In [5]:
# if __name__ == "__main__":
#     #songs = ['drwn','strfl']
#     songs = ['wu2s','drwn','strfl']
#     sides = ['left','right','center']
#     time_top_start = time.time()
#     for song in songs:
#         for side in sides:
#             top_proc_start = time.time()
#             main(song, side)
#             proc_time = time.time() - top_proc_start
#             print(f'Processed {song}_{side}. Took {int(proc_time/60)}:{int(proc_time%60)}')
#     time_top_start = time_top_start - time.time()
#     print(f'Processed {song}_{side}. Took {int(time_top_start/60)}:{int(time_top_start%60)}')
#     print("Done XD")

In [ ]:
if __name__ == "__main__":
    #songs = ['drwn','strfl']
    songs = ['wu2s','drwn','strfl']
    sides = ['center','left','right']
    time_top_start = time.time()
    song = songs[1]
    for side in sides:
        top_proc_start = time.time()
        main(song, side)
        proc_time = time.time() - top_proc_start
        print(f'Processed {song}_{side}. Took {int(proc_time/60)}:{int(proc_time%60)}')
    time_top_start = time_top_start - time.time()
    print(f'Processed {song}_{side}. Took {int(time_top_start/60)}:{int(time_top_start%60)}')
    print("Done XD")

Time: 0: 0 (0.02%)
ETA: 12:49 [0/6525]
Time: 0:37 (7.68%)
ETA: 7:34 [500/6525]
Time: 1:14 (15.34%)
ETA: 6:49 [1000/6525]
Time: 1:49 (23.00%)
ETA: 6:7 [1500/6525]
Time: 2:24 (30.67%)
ETA: 5:26 [2000/6525]
Time: 2:59 (38.33%)
ETA: 4:49 [2500/6525]
Time: 3:32 (45.99%)
ETA: 4:9 [3000/6525]
Time: 4: 8 (53.66%)
ETA: 3:34 [3500/6525]
Time: 4:45 (61.32%)
ETA: 2:59 [4000/6525]
Time: 5:27 (68.98%)
ETA: 2:27 [4500/6525]
Time: 6: 4 (76.64%)
ETA: 1:51 [5000/6525]
Time: 6:38 (84.31%)
ETA: 1:14 [5500/6525]
Time: 7:11 (91.97%)
ETA: 0:37 [6000/6525]
Time: 8: 4 (99.63%)
ETA: 0:1 [6500/6525]
Schemeing...
0 bad frames (0.00)
Time: 8: 6
Scheme complete >:)
Processed drwn_center. Took 13:21
Time: 0: 0 (0.02%)
ETA: 62:35 [0/6497]
Time: 0:33 (7.71%)
ETA: 6:35 [500/6497]
Time: 1: 6 (15.41%)
ETA: 6:3 [1000/6497]
Time: 2:13 (23.10%)
ETA: 7:25 [1500/6497]
Time: 3: 2 (30.80%)
ETA: 6:49 [2000/6497]
Time: 3:36 (38.49%)
ETA: 5:45 [2500/6497]
Time: 5: 0 (46.19%)
ETA: 5:50 [3000/6497]
Time: 5:36 (53.89%)
ETA: 4:48 [350